In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:

from tqdm.notebook import tqdm

from extract_features import *
from concurrent.futures import ProcessPoolExecutor, as_completed

In [21]:
def find_max_duration(cardiac_states):
    state_starts = cardiac_states[0]
    state_ends = cardiac_states[1]
    no_cycles = state_starts.shape[1]

    max_duration = -1

    for i in range(no_cycles):
        if np.any(np.isnan([state_ends[3, i], state_starts[0, i]])):
            continue
        
        cur_duration = state_ends[3, i] - state_starts[0, i]
        tqdm.write(str(cur_duration))
        
        if cur_duration > max_duration:
            max_duration = cur_duration

    return max_duration

In [11]:
data_folder = "C:/Users/lumin/Desktop/Work/20212/Data/physionet.org/files/circor-heart-sound/1.0.3/training_data"

# Find the patient data files.
patient_files = find_patient_files(data_folder)
num_patient_files = len(patient_files)

# Create a executor with 4 workers
executor = ProcessPoolExecutor(max_workers=4)
inputs = []

# max_dur = 0
max_cycles_no = 0

for i in tqdm(range(num_patient_files), desc="Patients", position=0):
    # Load the current patient data and recordings.
    current_patient_data = load_patient_data(patient_files[i])
    current_recordings_paths, current_recordings = load_recordings(data_folder, current_patient_data, preprocess=False)

    for j in range(len(current_recordings)):
        # if len(current_recordings[j]) > max_dur:
        #     max_dur = len(current_recordings[j])
        #     tqdm.write(f'Current max duration: {max_dur}')

        segmentation_file_path = AudioUtil.get_segmentation_file(current_recordings_paths[j])
        cardiac_states = AudioUtil.get_cardiac_states(segmentation_file_path)

        if cardiac_states is None:
            continue
        
        cardiac_cycles = AudioUtil.split_cardiac_cycles(current_recordings_paths[j], cardiac_states)


        if len(cardiac_cycles) > max_cycles_no:
            max_cycles_no = len(cardiac_cycles)
            tqdm.write(f'Current max cycle no: {max_cycles_no}')



# total_max_duration = -1
# futures = [executor.submit(get_cardiac_states, input) for input in inputs]
# for future in as_completed(futures):
#     cardiac_states = future.result()
#     if cardiac_states is None:
#         continue
#     cur_max_duration = find_max_duration(cardiac_states)
#     if cur_max_duration > total_max_duration:
#         total_max_duration = cur_max_duration
#         tqdm.write(f'Current maximum duration is: {total_max_duration}')

tqdm.write(f'max cycle duration is {max_cycles_no}')

Patients:   0%|          | 0/942 [00:00<?, ?it/s]

Current max cycle no: 17
Current max cycle no: 24
Current max cycle no: 42
Current max cycle no: 43
Current max cycle no: 46
Current max cycle no: 47
Current max cycle no: 66
Current max cycle no: 78
Current max cycle no: 93
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\lumin\anaconda3\envs\py38\lib\site-packages\IPython\core\interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\lumin\AppData\Local\Temp\ipykernel_25056\1478479149.py", line 26, in <cell line: 14>
    cardiac_cycles = AudioUtil.split_cardiac_cycles(current_recordings_paths[j], cardiac_states)
  File "c:\Users\lumin\Desktop\Work\20212\source-code\python-classifier-2022\audio_util.py", line 223, in split_cardiac_cycles
    state_starts = cardiac_states[0]
TypeError: 'NoneType' object is not subscriptable

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "c:\Users\lumin\anaconda3\envs\py38\lib\site-packages\IPython\core\interactiveshell.py", line 1993, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "c:\Users\lumin\anaconda3\envs\py38\lib\site-packages\IPython\core\ultratb.py", line 1118, i

Set 2.5sec as max_duration

Set 60s as max_recording_duration